In [1]:
import baostock as bs
import pandas as pd

# login
lg = bs.login()

# obtain the historical data of CSI 300
rs = bs.query_history_k_data_plus("sh.000300",  
    "date,code,open,high,low,close,volume",
    start_date='2015-08-01', end_date='2022-07-31',
    frequency="d", adjustflag="3")

# output
data_list = []
while (rs.error_code == '0') & rs.next():
    # Get a record and merge the records together
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)

# 结Output result sets to csv file
#result.to_csv("szzz.csv", index=False)
print(result)

# logout
bs.logout()

login success!
            date       code       open       high        low      close  \
0     2015-08-03  sh.000300  3766.0720  3829.2350  3742.5780  3829.2350   
1     2015-08-04  sh.000300  3827.3730  3948.4490  3801.0280  3948.1560   
2     2015-08-05  sh.000300  3937.6230  3962.2770  3858.6600  3866.9000   
3     2015-08-06  sh.000300  3802.9280  3888.6930  3793.2370  3831.8520   
4     2015-08-07  sh.000300  3866.4300  3926.6490  3857.9080  3906.9450   
...          ...        ...        ...        ...        ...        ...   
1696  2022-07-25  sh.000300  4233.9277  4234.8448  4201.9190  4212.6441   
1697  2022-07-26  sh.000300  4220.0911  4259.9854  4214.5343  4245.9758   
1698  2022-07-27  sh.000300  4236.1540  4242.0951  4217.2809  4225.0357   
1699  2022-07-28  sh.000300  4242.2743  4267.9181  4223.0290  4225.6733   
1700  2022-07-29  sh.000300  4226.4245  4243.5143  4160.8735  4170.1019   

           volume  
0     24580503000  
1     22023655600  
2     20048426900  
3   

In [2]:
result.dtypes

date      object
code      object
open      object
high      object
low       object
close     object
volume    object
dtype: object

In [3]:
total=result

In [4]:
# Converting data types
total["open"] = pd.to_numeric(total["open"],errors='coerce')
total["high"] = pd.to_numeric(total["high"],errors='coerce')
total["low"] = pd.to_numeric(total["low"],errors='coerce')
total["close"] = pd.to_numeric(total["close"],errors='coerce')
total["volume"] = pd.to_numeric(total["volume"],errors='coerce')


total.dtypes

date       object
code       object
open      float64
high      float64
low       float64
close     float64
volume      int64
dtype: object

In [5]:
#The target variable is obtained by the difference of "close" indicators

c=total['close'].values
print(type(c))

import numpy as np
u=np.diff(c)


d=np.append(u,[0]) 

total['variety']=d

total


#is used for prediction, so the last line is meaningless, delete it
#df=df.drop(total.tail(1),index)

#total.drop([len(total)-1],inplace=True)

total.drop(result.tail(1).index,inplace=True) 

#target=0,means down:target=1 means up

total['target'] =total['variety'].apply(lambda x: 1 if x > 0 else 0)
total.head()

total.dtypes

df=total.drop(['code','variety'],axis=1)


df.head()

<class 'numpy.ndarray'>


,date,open,high,low,close,volume,target
0,2015-08-03,3766.072,3829.235,3742.578,3829.235,24580503000,1
1,2015-08-04,3827.373,3948.449,3801.028,3948.156,22023655600,0
2,2015-08-05,3937.623,3962.277,3858.660,3866.900,20048426900,0
3,2015-08-06,3802.928,3888.693,3793.237,3831.852,14727869400,1
4,2015-08-07,3866.430,3926.649,3857.908,3906.945,18334915300,1


In [6]:
#save 
df.to_csv('20152022.csv')

In [101]:
#dataset1 the raw indicators
import pandas as pd
import numpy as np
df1 = pd.read_csv('20152022.csv',index_col=0)
df1.head()

,date,open,high,low,close,volume,target
0,2015-08-03,3766.072,3829.235,3742.578,3829.235,24580503000,1
1,2015-08-04,3827.373,3948.449,3801.028,3948.156,22023655600,0
2,2015-08-05,3937.623,3962.277,3858.660,3866.900,20048426900,0
3,2015-08-06,3802.928,3888.693,3793.237,3831.852,14727869400,1
4,2015-08-07,3866.430,3926.649,3857.908,3906.945,18334915300,1


In [102]:
df1.dtypes

date       object
open      float64
high      float64
low       float64
close     float64
volume      int64
target      int64
dtype: object

In [103]:
data=df1

In [104]:
#dataset2 the continous indicators
import talib 
import warnings
warnings.filterwarnings("ignore")

data["SMA10"]=talib.SMA(data['close'],timeperiod=10)
data['WMA10']=talib.MA(data['close'],timeperiod=10,matype=2)

data['macd'], data['macdsignal'], data['macdhist'] = talib.MACD(data['close'], fastperiod=12, slowperiod=26, signalperiod=9)

data['stochK'], data['stochD'] = talib.STOCH(data['high'].values, data['low'].values, data['close'].values, fastk_period=9, slowk_period=3,
                               slowk_matype=0, slowd_period=3, slowd_matype=0)


data['RSI'] = talib.RSI(data['close'], timeperiod=10)

data['MOM'] = talib.MOM(data['close'], timeperiod=10)


data['CCI'] = talib.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=10)


data['WILLR'] = talib. WILLR(data['high'].values, data['low'].values, data['close'].values, timeperiod=10)


data['ADX'] = talib. ADX(data['high'].values,data['low'].values, data['close'].values, timeperiod=10)

data.head(50)


data=data.drop(['date','open','low','high','volume','macdsignal','macdhist'], axis='columns')

df2=data.drop(data.head(33).index,inplace=False)
df11=df2.drop(['close'],axis='columns')


df11.head()


,target,SMA10,WMA10,macd,stochK,stochD,RSI,MOM,CCI,WILLR,ADX
33,1,3297.7208,3278.505000,-196.258349,45.426673,41.753857,46.660675,57.762,-19.003761,-40.098753,34.067767
34,0,3298.2215,3286.015764,-177.049335,59.432889,50.133078,48.891648,5.007,55.270235,-29.673768,31.272512
35,1,3284.5938,3279.616945,-166.044719,63.328824,56.062795,43.859202,-136.277,-13.513826,-50.679913,29.572068
36,0,3277.3376,3279.690618,-153.777950,64.474690,62.412134,45.655916,-72.562,8.577902,-38.978964,28.041667
37,1,3265.8139,3271.438509,-146.646527,52.511375,60.104963,42.045964,-115.237,-61.770786,-58.532020,27.556152


In [105]:
df2.head()

,close,target,SMA10,WMA10,macd,stochK,stochD,RSI,MOM,CCI,WILLR,ADX
33,3308.247,1,3297.7208,3278.505000,-196.258349,45.426673,41.753857,46.660675,57.762,-19.003761,-40.098753,34.067767
34,3339.030,0,3298.2215,3286.015764,-177.049335,59.432889,50.133078,48.891648,5.007,55.270235,-29.673768,31.272512
35,3263.028,1,3284.5938,3279.616945,-166.044719,63.328824,56.062795,43.859202,-136.277,-13.513826,-50.679913,29.572068
36,3284.999,0,3277.3376,3279.690618,-153.777950,64.474690,62.412134,45.655916,-72.562,8.577902,-38.978964,28.041667
37,3231.951,1,3265.8139,3271.438509,-146.646527,52.511375,60.104963,42.045964,-115.237,-61.770786,-58.532020,27.556152


In [106]:
data=df2

In [107]:
#dataset3 discreted continous indicator,that is,signal indicators
data["SSMA10"] = data[["SMA10", "close"]].apply(lambda x: x["close"] - x["SMA10"], axis=1)
data.head()

data['SSMA10'] = data['SSMA10'].apply(lambda x: 1 if x > 0 else -1)
data.head()

data["WWMA10"] = data[["WMA10", "close"]].apply(lambda x: x["close"] - x["WMA10"], axis=1)
data.head()

data['WWMA10'] = data['WWMA10'].apply(lambda x: 1 if x > 0 else -1)
data.head()

data['MMOM'] = data['MOM'].apply(lambda x: 1 if x >= 0 else -1)
data.head()

import numpy as np

c=data['stochK'].values
print(type(c))


u=np.diff(c)

#为了对齐方便合并df
d=np.append(u,[0]) 

data['stochKK']=d

data.head(10)




#It's used for prediction, so the last line is meaningless. Delete it
#df=df.drop(total.tail(1),index)

data.drop([len(data)-1],inplace=True)

#target=0,means down:target=1 means up

data['stochKK'] =data['stochKK'].apply(lambda x: 1 if x > 0 else -1)
data.head(10)

import numpy as np

c=data['stochD'].values
print(type(c))


u=np.diff(c)


d=np.append(u,[0]) 

data['stochDD']=d

data.head(10)






#df=df.drop(total.tail(1),index)

data.drop([len(data)-1],inplace=True)

#target=0,means down:target=1 means up

data['stochDD'] =data['stochDD'].apply(lambda x: 1 if x > 0 else -1)
data.head(10)



c=data['macd'].values
print(type(c))


u=np.diff(c)

#
d=np.append(u,[0]) 

data['macdd']=d

data.head(10)









#df=df.drop(total.tail(1),index)

data.drop([len(data)-1],inplace=True)

#target=0,means down:target=1 means up

data['macdd'] =data['macdd'].apply(lambda x: 1 if x > 0 else -1)
data.head(10)



import numpy as np

c=data['WILLR'].values
print(type(c))
u=np.diff(c)


d=np.append(u,[0]) 

data['WILLRR']=d

data.head(10)

data.drop([len(data)-1],inplace=True)

#target=0,means down:target=1 means up

data['WILLRR'] =data['WILLRR'].apply(lambda x: 1 if x > 0 else -1)
data.head(10)

import numpy as np

c=data['ADX'].values
print(type(c))
u=np.diff(c)

d=np.append(u,[0]) 

data['ADXX']=d

data.head(10)


data.drop([len(data)-1],inplace=True)

#target=0,means down:target=1 means up

data['ADXX'] =data['ADXX'].apply(lambda x: 1 if x > 0 else -1)
data.head(10)

data['RSI1'] =data['RSI'].apply(lambda x: -1 if x > 70 else(1 if x <30   else  0))
#data['RSI','RSI1'].head(20)

import numpy as np

c=data['RSI'].values
print(type(c))
u=np.diff(c)


d=np.append(u,[0]) 

data['RSI2']=d

data.head(10)

data.drop([len(data)-1],inplace=True)

#target=0,means down:target=1 means up

data['RSI2'] =data['RSI2'].apply(lambda x: 1 if x > 0 else -1)
data.head(20)




data['CCI2'] =data['CCI'].apply(lambda x: -1 if x > 200 else(1 if x <-200   else  0))
#data['RSI','RSI1'].head(20)

c=data['CCI'].values
print(type(c))
u=np.diff(c)


d=np.append(u,[0]) 

data['CCII']=d

data.head(10)

data.drop([len(data)-1],inplace=True)

#target=0,means down:target=1 means up

data['CCII'] =data['CCII'].apply(lambda x: 1 if x > 0 else -1)
data.head(20)

df22=data[['target','SSMA10','WWMA10','MMOM','stochKK','stochDD','macdd','WILLRR','ADXX','RSI1','CCI2']]
df22.head()

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


,target,SSMA10,WWMA10,MMOM,stochKK,stochDD,macdd,WILLRR,ADXX,RSI1,CCI2
33,1,1,1,1,1,1,1,1,-1,0,0
34,0,1,1,1,1,1,1,-1,-1,0,0
35,1,-1,-1,-1,1,1,1,1,-1,0,0
36,0,1,1,-1,-1,-1,1,-1,-1,0,0
37,1,-1,-1,-1,-1,-1,1,1,-1,0,0


In [114]:
#dataset4 add sentiment indicator on the basis of dataset 3
# Construct sentiment indicator
import pandas as pd
#comments = pd.read_excel('0003002122.xlsx')
df = pd.read_excel('crawl.xls')
df.head()


df.shape[0]

import pandas as pd
#comments = pd.read_excel('0003002122.xlsx')
df1 = pd.read_csv('cpbaidu.csv')
df1.head()

df1.shape[0]

df2=pd.concat([df,df1],axis=1)
df2.head()

df3=df2[['read','mark','title','last_time','contentsenti']]

df3.shape[0]

df3.head()

df3['date']=df3['last_time'].str[:10]
df3.head(10)

df3['sentiment'] = df3['contentsenti'].map(lambda x: x-1)
df3.head(10)

df3.info()

df3['read'] = pd.to_numeric(df3['read'],errors='coerce')

df3.info()

#Descending order processing
df3=df3.sort_values(by='last_time',ascending=False,axis=0)


df3.head(10)

df3['sentiment'].apply(lambda x: float(x))
df3['read'].apply(lambda x: float(x))

import numpy as np
import math
df3['Wread']=df3['read'].apply(lambda x:np.log(math.e+x))

df3['WetghtSentiment']=df3['Wread']*df3['sentiment']

df3.head()



reverse_series = df3['date'].reindex(df3['date'].index[::-1])
datese=reverse_series.unique()

WetghtSentimentbyday = df3['WetghtSentiment'].groupby(df3['date']).sum()
Wreadbyday = df3['Wread'].groupby(df3['date']).sum()
datese=reverse_series.unique()
markbyday = pd.DataFrame()
markbyday['read']=Wreadbyday
markbyday['WetghtSentiment']=WetghtSentimentbyday
markbyday['date']=datese
markbyday['SentimentIndex']=markbyday['WetghtSentiment']/markbyday['read']
markbyday.head()







markbyday

markbyday.tail(76)

markbyday.drop(markbyday.tail(76).index)





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32599 entries, 0 to 32598
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   read          32599 non-null  object
 1   mark          32599 non-null  int64 
 2   title         32599 non-null  object
 3   last_time     32599 non-null  object
 4   contentsenti  32599 non-null  int64 
 5   date          32599 non-null  object
 6   sentiment     32599 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 1.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32599 entries, 0 to 32598
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   read          32595 non-null  float64
 1   mark          32599 non-null  int64  
 2   title         32599 non-null  object 
 3   last_time     32599 non-null  object 
 4   contentsenti  32599 non-null  int64  
 5   date          32599 non-null  object 
 6   sentime

,read,WetghtSentiment,date,SentimentIndex
date,,,,
2015-08-03,18.382065,5.438243,2015-08-03,0.295845
2015-08-05,6.388088,0.000000,2015-08-05,0.000000
2015-08-06,36.047353,-12.364291,2015-08-06,-0.343001
2015-08-07,18.499832,-18.499832,2015-08-07,-1.000000
2015-08-08,6.739003,-6.739003,2015-08-08,-1.000000
...,...,...,...,...
2022-07-27,74.230875,-0.967079,2022-07-27,-0.013028
2022-07-28,131.382147,17.607976,2022-07-28,0.134021
2022-07-29,118.199300,-27.957522,2022-07-29,-0.236529


In [116]:
##dataset4 add sentiment indicator on the basis of dataset 3
import baostock as bs
import pandas as pd

# login
lg = bs.login()

# obtain the historical data of CSI 300
rs = bs.query_history_k_data_plus("sh.000300",  
    "date,open,high,low,close,volume",
    start_date='2015-08-03', end_date='2022-07-31',
    frequency="d", adjustflag="3")

# output
data_list = []
while (rs.error_code == '0') & rs.next():

    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)


#result.to_csv("szzz.csv", index=False)
print(result)

# logout
bs.logout()


result.head()
markbyday.head()
total = result.set_index('date').join(markbyday.set_index('date'),how='left')
total

values = {'SentimentIndex': 0,'WetghtSentiment':0,'read':0}
total.fillna(value=values,inplace=True)


total.info()

total["open"] = pd.to_numeric(total["open"],errors='coerce')
total["high"] = pd.to_numeric(total["high"],errors='coerce')
total["low"] = pd.to_numeric(total["low"],errors='coerce')
total["close"] = pd.to_numeric(total["close"],errors='coerce')
total["volume"] = pd.to_numeric(total["volume"],errors='coerce')
total.dtypes

print(total[total.isnull().T.any()])

df=total

df.shape[0]

c=df['close'].values
print(type(c))

import numpy as np
u=np.diff(c)


d=np.append(u,[0]) 

df['variety']=d

df.tail()


df=df.drop(total.tail(1).index)


#df.drop([len(df)-1],inplace=True)

#target=0,means down:target=1 means up



df.tail()

df.shape[0]

#target=0,means down:target=1 means up

df['target'] =df['variety'].apply(lambda x: 1 if x > 0 else 0)
df.head()


df1=df.drop(['read','WetghtSentiment','variety'],axis=1)
df1

print("df1.label",df1.target.value_counts())

#data=df1.drop(['amount','preclose','turn','pctChg'],axis=1)
#data

data=df1

import warnings
warnings.filterwarnings("ignore")
import talib 

data["SMA10"]=talib.SMA(data['close'],timeperiod=10)
data['WMA10']=talib.MA(data['close'],timeperiod=10,matype=2)

data['macd'], data['macdsignal'], data['macdhist'] = talib.MACD(data['close'], fastperiod=12, slowperiod=26, signalperiod=9)

data['stochK'], data['stochD'] = talib.STOCH(data['high'].values, data['low'].values, data['close'].values, fastk_period=9, slowk_period=3,
                               slowk_matype=0, slowd_period=3, slowd_matype=0)


data['RSI'] = talib.RSI(data['close'], timeperiod=10)

data['MOM'] = talib.MOM(data['close'], timeperiod=10)


data['CCI'] = talib.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=10)


data['WILLR'] = talib. WILLR(data['high'].values, data['low'].values, data['close'].values, timeperiod=10)


data['ADX'] = talib. ADX(data['high'].values,data['low'].values, data['close'].values, timeperiod=10)

CLOSE=data.close.values

data = data.reset_index(drop=True)

data.shape[0]

data.head(50)

data=data.drop(data.head(33).index,inplace=False)


data.head()

data["SSMA10"] = data[["SMA10", "close"]].apply(lambda x: x["close"] - x["SMA10"], axis=1)
data.head()

data['SSMA10'] = data['SSMA10'].apply(lambda x: 1 if x > 0 else -1)
data.head()

data["WWMA10"] = data[["WMA10", "close"]].apply(lambda x: x["close"] - x["WMA10"], axis=1)
data.head()

data['WWMA10'] = data['WWMA10'].apply(lambda x: 1 if x > 0 else -1)
data.head()

data['MMOM'] = data['MOM'].apply(lambda x: 1 if x >= 0 else -1)
data.head()

import numpy as np

c=data['stochK'].values
print(type(c))


u=np.diff(c)


d=np.append(u,[0]) 

data['stochKK']=d

data.head(10)




#df=df.drop(total.tail(1),index)

data.drop([len(data)-1],inplace=True)

#target=0,means down:target=1 means up

data['stochKK'] =data['stochKK'].apply(lambda x: 1 if x > 0 else -1)
data.head(10)

import numpy as np

c=data['stochD'].values
print(type(c))


u=np.diff(c)

d=np.append(u,[0]) 

data['stochDD']=d

data.head(10)






#df=df.drop(total.tail(1),index)

data.drop([len(data)-1],inplace=True)

#target=0,means down:target=1 means up

data['stochDD'] =data['stochDD'].apply(lambda x: 1 if x > 0 else -1)
data.head(10)

import numpy as np

c=data['macd'].values
print(type(c))


u=np.diff(c)


d=np.append(u,[0]) 

data['macdd']=d

data.head(10)








#df=df.drop(total.tail(1),index)

data.drop([len(data)-1],inplace=True)

#target=0,means down:target=1 means up

data['macdd'] =data['macdd'].apply(lambda x: 1 if x > 0 else -1)
data.head(10)

import numpy as np

c=data['WILLR'].values
print(type(c))
u=np.diff(c)


d=np.append(u,[0]) 

data['WILLRR']=d

data.head(10)




#df=df.drop(total.tail(1),index)

data.drop([len(data)-1],inplace=True)



data['WILLRR'] =data['WILLRR'].apply(lambda x: 1 if x > 0 else -1)
data.head(10)

import numpy as np

c=data['ADX'].values
print(type(c))
u=np.diff(c)


d=np.append(u,[0]) 

data['ADXX']=d

data.head(10)


data.drop([len(data)-1],inplace=True)

#target=0,means down:target=1 means up

data['ADXX'] =data['ADXX'].apply(lambda x: 1 if x > 0 else -1)
data.head(10)

data['RSI1'] =data['RSI'].apply(lambda x: -1 if x > 70 else(1 if x <30   else  0))
#data['RSI','RSI1'].head(20)

import numpy as np

c=data['RSI'].values
print(type(c))
u=np.diff(c)


d=np.append(u,[0]) 

data['RSI2']=d

data.head(10)

data.drop([len(data)-1],inplace=True)

#target=0,means down:target=1 means up

data['RSI2'] =data['RSI2'].apply(lambda x: 1 if x > 0 else -1)
data.head(20)

data['CCI2'] =data['CCI'].apply(lambda x: -1 if x > 200 else(1 if x <-200   else  0))
#data['RSI','RSI1'].head(20)

c=data['CCI'].values
print(type(c))
u=np.diff(c)


d=np.append(u,[0]) 

data['CCII']=d

data.head(10)

data.drop([len(data)-1],inplace=True)

#target=0,means down:target=1 means up

data['CCII'] =data['CCII'].apply(lambda x: 1 if x > 0 else -1)
data.head(20)




df1 = data.astype({'SSMA10':'str','WWMA10':'str','MMOM':'str','stochKK':'str','stochDD':'str','macdd':'str','WILLRR':'str','ADXX':'str','RSI1':'str','CCI2':'str'})
#'RSI1':'str','CCI2':'str'

df1.dtypes

features = ["SSMA10", "WWMA10", "MMOM", "stochKK","stochDD","macdd","WILLRR","ADXX","RSI1","CCI2"]
 # Filter out categorical variables for modeling
df2 = pd.get_dummies(df1[features])

    #"RSI1","CCI2"

df2

df3 = pd.concat([df1['SentimentIndex'],df2],axis=1)
df3

df4 = pd.concat([df1['target'],df3],axis=1)
df4

login success!
            date       open       high        low      close       volume
0     2015-08-03  3766.0720  3829.2350  3742.5780  3829.2350  24580503000
1     2015-08-04  3827.3730  3948.4490  3801.0280  3948.1560  22023655600
2     2015-08-05  3937.6230  3962.2770  3858.6600  3866.9000  20048426900
3     2015-08-06  3802.9280  3888.6930  3793.2370  3831.8520  14727869400
4     2015-08-07  3866.4300  3926.6490  3857.9080  3906.9450  18334915300
...          ...        ...        ...        ...        ...          ...
1696  2022-07-25  4233.9277  4234.8448  4201.9190  4212.6441   8363666400
1697  2022-07-26  4220.0911  4259.9854  4214.5343  4245.9758   8582327400
1698  2022-07-27  4236.1540  4242.0951  4217.2809  4225.0357   7890494000
1699  2022-07-28  4242.2743  4267.9181  4223.0290  4225.6733  10753193000
1700  2022-07-29  4226.4245  4243.5143  4160.8735  4170.1019  11417928600

[1701 rows x 6 columns]
logout success!
<class 'pandas.core.frame.DataFrame'>
Index: 1701 entrie

,target,SentimentIndex,SSMA10_-1,SSMA10_1,WWMA10_-1,WWMA10_1,MMOM_-1,MMOM_1,stochKK_-1,stochKK_1,...,WILLRR_-1,WILLRR_1,ADXX_-1,ADXX_1,RSI1_-1,RSI1_0,RSI1_1,CCI2_-1,CCI2_0,CCI2_1
33,1,0.428590,0,1,0,1,0,1,0,1,...,0,1,1,0,0,1,0,0,1,0
34,0,-0.483222,0,1,0,1,0,1,0,1,...,1,0,1,0,0,1,0,0,1,0
35,1,-1.000000,1,0,1,0,1,0,0,1,...,0,1,1,0,0,1,0,0,1,0
36,0,-0.204933,0,1,0,1,1,0,1,0,...,1,0,1,0,0,1,0,0,1,0
37,1,-1.000000,1,0,1,0,1,0,1,0,...,0,1,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,0,-0.221616,1,0,1,0,1,0,1,0,...,1,0,0,1,0,1,0,0,1,0
1696,1,0.192939,1,0,1,0,1,0,0,1,...,0,1,1,0,0,1,0,0,1,0
1697,0,-0.119951,1,0,1,0,1,0,1,0,...,1,0,1,0,0,1,0,0,1,0
1698,1,-0.013028,1,0,1,0,1,0,0,1,...,0,1,1,0,0,1,0,0,1,0
